# Intro & Instructions

Jupyter notebook for data extraction and processing for Joint Treasury data analysis. **Execution is on Colab** (not locally).

Setup is done in the first section in order to have proper config for the whole nobtebook.

# Setup

In [ ]:
"""
Setup all the required variables & logic for the notebook.
"""
%%capture  # To clear the output from the cell

# ==============================================
#  Install Required Packages
# ==============================================

! pip install dune_spice  # Install dune_spice for querying data from Dune Analytics
# !pip install dune_client
# !pip install dataclasses_json

# ==============================================
#  Import Required Libraries
# ==============================================

# Google authentication libraries
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Other libraries
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import spice

import pandas as pd
import polars
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pprint
import requests
from datetime import datetime
import time
import os

# ==============================================
#  Configuration Class
# ==============================================

class Config:
    """Configuration class containing all constants and settings for the script."""
    
    # Environment Variables and API Keys
    DUNE_API_KEY: str = 'RyXNIYLH4uE5NeEjLWQBZEcrkjTRw2EH'
    COINGECKO_API_KEY: str = 'CG-jN5KXD1QFHacbpJb3T7PVJ3P'
    
    # API Endpoints
    COINGECKO_BASE_URL: str = "https://api.coingecko.com/api/v3"
    COINGECKO_PRICE_ENDPOINT: str = f"{COINGECKO_BASE_URL}/simple/price?vs_currencies=usd&ids="
    
    # File Paths and Directories
    DATA_DIR: str = "./data"
    
    # Google Sheets Configuration
    WORKBOOK_URL: str = "https://docs.google.com/spreadsheets/d/1mIQTla9L7l3FBh1k8xtpHQwHMN4D7nHl1u5nsdeWdA0/"
    
    # Important Dates
    TTE_DATE: str = "2024-04-23"
    ETL_NOW: datetime = pd.to_datetime(datetime.now())
    
    # API Request Settings
    MAX_RETRIES: int = 3
    DEFAULT_TIMEOUT: int = 10
    RETRY_DELAY: int = 5
    
    # Token Configurations
    PORTFOLIO_TOKENS = {
        'coingecko_id': {
            # Stablecoins
            'dai': 'dai',
            'xdai': 'xdai',
            'usdc': 'usd-coin',
            'usdt': 'tether',
            # ETH
            'eth': 'ethereum',
            # SAFE & GNO
            'safe': 'safe',
            'gno': 'gnosis',
            # Other Tokens
            'aura': 'aura-finance',
            'bal': 'balancer'
        },
        'asset_class': {
            'dai': 'stablecoins',
            'xdai': 'stablecoins',
            'usdc': 'stablecoins',
            'usdt': 'stablecoins',
            'eth': 'eth',
            'weth': 'eth',
            'safe': 'safe',
            'gno': 'gno',
            'aura': 'other',
            'bal': 'other'
        },
        'similar_tokens': {  # Coingecko ID for similar tokens (to compare market data)
            'safe': 'safe',
            'gno': 'gnosis',
            'eth': 'ethereum',
            'uni': 'uniswap',
            'link': 'chainlink',
            'grt': 'the-graph'
        },
        'token_class': {  # mapper from token to asset for matching with current price
            'SAFE': 'safe',
            'GNO': 'gno',
            'WETH': 'eth',
            'USDC': 'usdc'
        }
    }
    
    # Token Address Mappings
    TOKEN_ADDRESSES = {
        '0X5AFE3855358E112B5647B952709E6165E1C1EEEE': 'SAFE',  # Ethereum
        '0X4D18815D14FE5C3304E87B3FA18318BAA5C23820': 'SAFE',  # Gnosis
        '0XC02AAA39B223FE8D0A0E5C4F27EAD9083C756CC2': 'ETH',  # Ethereum WETH
        '0X9C58BACC331C9AA871AFD802DB6379A98E80CEDB': 'GNO',  # Gnosis
        '0XA0B86991C6218B36C1D19D4A2E9EB0CE3606EB48': 'USDC',  # Ethereum
        '0X5AFE3855358E112B5647B952709E6165E1C1EEEE-0XC02AAA39B223FE8D0A0E5C4F27EAD9083C756CC2': 'SAFE-ETH',  # Ethereum
        '0X5AFE3855358E112B5647B952709E6165E1C1EEEE-0XA0B86991C6218B36C1D19D4A2E9EB0CE3606EB48': 'SAFE-USDC',  # Ethereum
        '0X4D18815D14FE5C3304E87B3FA18318BAA5C23820-0X9C58BACC331C9AA871AFD802DB6379A98E80CEDB': 'SAFE-GNO'  # Gnosis
    }
    
    # Plot Settings
    @staticmethod
    def setup_plot_style():
        plt.style.use('seaborn-v0_8')
        sns.set_theme(style="darkgrid")

# ==============================================
#  Create directory for local/colab execution
# ==============================================

# Create the data directory
os.makedirs(Config.DATA_DIR, exist_ok=True)

# Setup plot style
Config.setup_plot_style()

# Initialize pretty printer
pp = pprint.PrettyPrinter(indent=4)

# Open workbook
wb = gc.open_by_url(Config.WORKBOOK_URL)

# ==============================================================================
# Function definitions
# ==============================================================================

# Directly generate df from gsheet imports

def etl_gen_df_from_gsheet(wb_url, page, output_type = 'json', index_col = ''):
  """Generates dataframe from a table stored in google sheets
  str dtype is forced for conversion to decimal.Decimal type with all decimals

  Args:
    wb_url (str): url of the spreadsheet
    page (str): name of the page that has to be accessed
    output_type (str): type of output desired (json/df)
    index_col (str): col_label to be used as string

  Returns:
    df (DataFrame): containing information on the page
  """

  wb = gc.open_by_url(wb_url)

  sheet = wb.worksheet(page)
  records = sheet.get_all_records()

  if output_type == 'df':
    df = pd.DataFrame(records, dtype = str)
    if index_col == '':
      pass
    else:
      df.set_index(index_col, drop = True, inplace = True)

    output = df

  else:

    output = records

  return output

# DATA - Ext

Extracting and handling data to be exported to the JT Sheet


## CG - prices

In [ ]:
"""
Seeks current prices for portfolio tokens or other tokens of interest - CoinGecko API
"""

# ==============================================================================
# Request construction
# ==============================================================================

# Build the string that contains all token IDs to make the API call
id_string = ''
for id in Config.PORTFOLIO_TOKENS['coingecko_id']:
    id_string = id_string + Config.PORTFOLIO_TOKENS['coingecko_id'][id] + '%2C'

id_string = id_string[:-3]  # Exclude the last '%2C' from the call - it makes it crash

params = {
    'x_cg_demo_api_key': Config.COINGECKO_API_KEY
}

r = requests.get(Config.COINGECKO_PRICE_ENDPOINT + id_string, params=params)

# ==============================================================================
# Request handling
# ==============================================================================

df = pd.DataFrame(r.json())

# Mapping from coingecko ids to asset
inv_map = {v: k for k, v in Config.PORTFOLIO_TOKENS['coingecko_id'].items()}

# Rename the columns from coingecko_id to symbol
df = df.rename(columns=inv_map).T
df.columns = ['price']

# Add the asset class for each asset - this is used later in the JT sheet for asset wide calculations
df['asset_class'] = df.index.map(Config.PORTFOLIO_TOKENS['asset_class'])

# Add specific cases listed in the dictionary in the setup index
df.loc['weth', :] = df.loc['eth', :]

# Add datetime + etl_dt (etl datetime for reference of when info was fetched)
df['datetime'] = Config.ETL_NOW
df['datetime'] = df['datetime'].dt.floor('s')
df['etl_dt'] = df['datetime']

# ==============================================================================
# Data export
# ==============================================================================

prices_df = df.copy()

# Open sheet inside workbook
sheet = wb.worksheet('prices')

# Convert relevant columns in dataframe to str to export
prices_df = prices_df.astype({'datetime': str, 'etl_dt': str})

# Reset index for all data to appear - symbols were the index
prices_df = prices_df.reset_index()

# DataFrame export
prices_df_export_list = [prices_df.columns.tolist()]  # Column labels
prices_df_export_list.extend(prices_df.values.tolist())
sheet.update("A1", prices_df_export_list)

# Export to csv
prices_df.to_csv(f"{Config.DATA_DIR}/prices.csv")

<ipython-input-7-1fd5175eb9f5>:68: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update("A1", prices_df_export_list)


## CG - tickers


In [ ]:
"""
Seeks current market data form CoinGecko - the table below each asset on their respective page
"""

# ==============================================================================
# Request construction
# ==============================================================================

asset = 'safe'

endpoint = f"{Config.COINGECKO_BASE_URL}/coins/{asset}/tickers?depth=true"

params = {
    'x_cg_demo_api_key': Config.COINGECKO_API_KEY
}

r = requests.get(endpoint, params=params)

# ==============================================================================
# Request handling
# ==============================================================================

tickers = r.json()['tickers']

data = []

for ticker in range(len(tickers)):
    data_temp = {
        'exchange': tickers[ticker]['market']['name'],
        'pair': tickers[ticker]['base'] + '-' + tickers[ticker]['target'],
        'price': tickers[ticker]['last'],
        'spread': tickers[ticker]['bid_ask_spread_percentage'],  # it's in percentage, have to pass to decimal
        '2_pct': tickers[ticker]['cost_to_move_up_usd'],
        '-2_pct': tickers[ticker]['cost_to_move_down_usd'],
        '24h_vol': tickers[ticker]['converted_volume']['usd'],
        'trust_score': tickers[ticker]['trust_score']
    }
    data.append(data_temp)

df = pd.DataFrame.from_records(data)

# Extract blockchain from DEXs by extracting it from the Exchange name
df['chain'] = df['exchange'].str.extract(r'\((.*?)\)', expand=False)

# Add tye of exchange (CEX/DEX)
df['type'] = df['chain'].apply(lambda x: 'DEX' if pd.notnull(x) else 'CEX')

# Add Volume percentage for each Pair
df['vol_pct'] = df['24h_vol'].div(df['24h_vol'].sum())

# Add datetime + etl_dt (etl datetime for reference of when info was fetched)
df['datetime'] = Config.ETL_NOW
df['datetime'] = df['datetime'].dt.floor('s')
df['etl_dt'] = df['datetime']

# Add vol*-2_pct - for volume weighted -2% size
df['depth_*_vol'] = df['-2_pct'] * df['24h_vol']

# Add Notes column to match the JT Sheet table that has notes
df['notes'] = ""

# Replace anomally signs in the 'pair' column
df['pair'] = df['pair'].str.replace('$', '')
df['pair'] = df['pair'].str.replace('SAFE1', 'SAFE')

# Replace token addresses
df['pair'] = df['pair'].replace(regex=Config.TOKEN_ADDRESSES)

# Replace NaN for "nan" for correct data for exporting to the Sheet - the same string that is fetched from the import
df['chain'] = df['chain'].fillna('nan')

# Order as the sheets table - it's to append below
df = df[['exchange', 'type', 'pair', 'price', 'spread', '2_pct', '-2_pct', '24h_vol', 'vol_pct', 'trust_score', 'chain', 'depth_*_vol', 'datetime', 'etl_dt', 'notes']]

# ==============================================================================
# Data export
# ==============================================================================

tickers_df = df.copy()

# Open sheet inside workbook
sheet = wb.worksheet('tickers')

# Convert dateitme in dataframe to str to export
tickers_df = tickers_df.astype({'datetime': str, 'etl_dt': str})

# Reset index for all data to appear
tickers_df = tickers_df.reset_index()

# Get current data from the tab (it's a timeseries that gets updated with latest data) - plus convert imported datatypes (only the relevant ones)
past_tickers_df = etl_gen_df_from_gsheet(wb_url=Config.WORKBOOK_URL, page='tickers', output_type='df')

# Convert data from all rows that should be numeric to numeric - convert to np.nan if impossible
past_tickers_df['spread'] = pd.to_numeric(past_tickers_df['spread'], errors='coerce')
past_tickers_df['2_pct'] = pd.to_numeric(past_tickers_df['2_pct'], errors='coerce')
past_tickers_df['vol_pct'] = pd.to_numeric(past_tickers_df['vol_pct'], errors='coerce')

# Convert all columns to their respective dtype for appending the new information
past_tickers_df = past_tickers_df.astype({
    'price': float,
    'spread': float,
    '2_pct': float,
    '2_pct': float,
    '24h_vol': float,
    'vol_pct': float,
    'depth_*_vol': float
})

# Join the historical and new dataframes
tickers_df = pd.concat([past_tickers_df, tickers_df], axis=0)  # Generate records dataframe to export

# Convert relevant columns in dataframe to str to export
tickers_df = tickers_df.astype({'spread': str, '2_pct': str, 'vol_pct': str, 'trust_score': str, 'chain': str, 'datetime': str, 'etl_dt': str})

# DataFrame export
tickers_df_export_list = [tickers_df.columns.tolist()]  # Column labels
tickers_df_export_list.extend(tickers_df.values.tolist())
sheet.update("A1", tickers_df_export_list)

# Export to csv
tickers_df.to_csv(f"{Config.DATA_DIR}/tickers.csv")

<ipython-input-8-9d03868f5851>:120: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update("A1", tickers_df_export_list)


InvalidJSONError: Out of range float values are not JSON compliant

## CG & Dune - market_data

- this can be optimized with a dune extractor

In [ ]:
"""
Seeks SAFE + similar tokens market data in CoinGecko (since the TTE)
Also SAFE DEX Volume to understand the share of trade in each
"""

# ==============================================================================
# ==============================================================================
# CoinGecko Call
# ==============================================================================
# ==============================================================================

# ==============================================================================
# Request construction & handling
# ==============================================================================

# Total number of days to look for market data (from tte to today)
days = datetime.today() - datetime.strptime(Config.TTE_DATE, "%Y-%m-%d")

data = []

for id in Config.PORTFOLIO_TOKENS['similar_tokens']:
    endpoint = f"{Config.COINGECKO_BASE_URL}/coins/{Config.PORTFOLIO_TOKENS['similar_tokens'][id]}/market_chart?"
    params = {
        'vs_currency': 'usd',
        'days': days.days + 2,  # Then for the shift() method to leave info on first day and have a buffer
        'interval': 'daily',
        'x_cg_demo_api_key': Config.COINGECKO_API_KEY
    }

    r = requests.get(endpoint, params=params)

    # Process price data
    price = pd.DataFrame(r.json()['prices'], columns=['date', 'price'])  # Opening price for the day (first day is not standard)
    price['date'] = pd.to_datetime(price['date'], unit='ms')
    price = price.set_index('date')
    price = price.resample('D', kind='period').last()

    # Process market cap data
    mcap = pd.DataFrame(r.json()['market_caps'], columns=['date', 'mcap'])  # Mcap for that price
    mcap['date'] = pd.to_datetime(mcap['date'], unit='ms')
    mcap = mcap.set_index('date')
    mcap = mcap.resample('D', kind = 'period').last()

    # Process volume data
    volume = pd.DataFrame(r.json()['total_volumes'], columns = ['date', 'volume'])  # Volume of the last day
    volume['date'] = pd.to_datetime(volume['date'], unit = 'ms')
    volume = volume.set_index('date')
    volume = volume.resample('D', kind = 'period').last()

    # Combine all data & generate asset attribute
    df = pd.concat([price, mcap, volume], axis = 'columns')
    df['asset'] = id

    # Adjust data to reflect the data consistently -  values for each day are actualy the finishing for the earlier one
    df.columns = ['open', 'mcap', 'volume', 'asset']
    df['volume'] = df['volume'].shift(-1)
    df['close'] = df['open'].shift(-1)
    df['mcap'] = df['mcap'].shift(-1)

    # Organize & slice for dates since TTE
    df = df[['asset', 'open', 'close', 'mcap', 'volume']]

    # Missing today's close so row is removed
    df = df.loc[tte_date:].iloc[:-1]

    data.append(df)
    print('Asset Price Fetched:', df['asset'].iloc[0].upper())
    time.sleep(3)

cg_df = pd.concat(data, axis = 0)

# Additional Calcs
cg_df['vol/mcap'] = cg_df['volume'] / cg_df['mcap']

# ==============================================================================
# ==============================================================================
# Dune API Call - SAFE DEX Data
# ==============================================================================
# ==============================================================================

# ==============================================================================
# Request construction & handling
# ==============================================================================

query_id = 3777854  # SAFE - * - GMD - $SAFE DEX Trading Volume (Pool)

df = spice.query(query_id, refresh=True)
query_result = df.to_pandas()

# Make date index to resample and SUM data
query_result['date'] = pd.to_datetime(query_result['block_date'])

# Daily aggregation of numeric data
dex_df = query_result.resample('D', kind = 'period', on = 'date').sum(numeric_only = True)

# Select & Rename columns
dex_df = dex_df[['trades', 'total_vol']]
dex_df.columns = ['dex_trades', 'dex_vol']

# ==============================================================================
# Joining CG + DEX Data & final things
# ==============================================================================

# Add asset col for joining
dex_df['asset'] = 'safe'

# DF Merge - the total_df DataFrame will also be used later
total_df = cg_df.merge(dex_df, how = 'left', on = ['date', 'asset'])

# CEX Volume Calculations
total_df['cex_vol'] = total_df['volume'] - total_df['dex_vol']

# Add etl_dt (etl datetime for reference of when info was fetched)
total_df['etl_dt'] = etl_now
total_df['etl_dt'] = total_df['etl_dt'].dt.floor('s')

# ==============================================================================
# Data export
# ==============================================================================

market_data_df = total_df.copy()

# Open sheet inside workbook
sheet = wb.worksheet('market_data')

# Reset index for all data to appear - symbols were the index
market_data_df = market_data_df.reset_index()

# Convert relevant columns in dataframe to str to export - plus additional NaN values to nan in string
market_data_df = market_data_df.astype({'date': str, 'etl_dt': str})
market_data_df = market_data_df.fillna('nan')

# DataFrame export
market_data_df_export_list = [market_data_df.columns.tolist()]  # Column labels
market_data_df_export_list.extend(market_data_df.values.tolist())

sheet.update("A1", market_data_df_export_list)

# Export to csv
market_data_df.to_csv('market_data.csv')

## Dune - dex_fees

In [ ]:
# """
# Execute and get results of the Dune Query for DEX Fees
# """

# # ==============================================================================
# # Request construction
# # ==============================================================================

# query_id = 3768733  # SAFE - * - GMD - $SAFE DEX Trading Fees

# df = spice.query(query_id, refresh=True)
# query_result = df.to_pandas()

# # ==============================================================================
# # Request handling
# # ==============================================================================

# # Multiplty by current price to get current value of fees (for comparison with Revert)
# query_result['fee_amount_usd_adj'] = query_result['token'].replace(regex = portfolio_tokens_v2['token_class'])  # replace token by the asset for price

# """Temporal Solution for Metis appearance
# """
# query_result = query_result.loc[query_result['fee_amount_usd_adj'] not in ['Metis', 'OLAS']]

# prices_ = prices_df.set_index('index')['price'].to_dict()  # get prices from the DataFrame

# query_result['fee_amount_usd_adj'] = query_result['fee_amount_usd_adj'].replace(regex = prices_).astype(float)  # replace with prices
# query_result['fee_amount_token'] = query_result['fee_amount_token'].replace('<nil>', 0)  # replace Dune nulls with 0
# query_result['fee_amount_usd_adj'] = query_result['fee_amount_usd_adj'] * query_result['fee_amount_token'].astype(float)  # multiply amount by price

# # Add etl_dt (etl datetime for reference of when info was fetched)
# query_result['etl_dt'] = etl_now
# query_result['etl_dt'] = query_result['etl_dt'].dt.floor('s')

# # ==============================================================================
# # Data export
# # ==============================================================================

# dex_fees = query_result.copy()

# # Open sheet inside workbook
# sheet = wb.worksheet('dex_fees')

# # Convert relevant columns in dataframe to str to export - plus additional NaN values to nan in string
# dex_fees = dex_fees.astype({'block_date': str, 'etl_dt': str})
# dex_fees = dex_fees.fillna('nan')

# # Reset index for all data to appear - symbols were the index
# dex_fees = dex_fees.reset_index()

# # DataFrame export
# dex_fees_export_list = [dex_fees.columns.tolist()]  # Column labels
# dex_fees_export_list.extend(dex_fees.values.tolist())
# sheet.update("A1", dex_fees_export_list)

# # Export to csv
# dex_fees.to_csv('dex_fees.csv')

## Dune - dex_trades

- Requires executing the Market Data script before
- Needed from query
  - block_date
  - blockchain
  - label
  - token_sold_symbol
  - token_bought_amount
  - token_sold_amount
  - amount_usd
  - tx_from

In [ ]:
"""
Seeks trading activity from Dune analytics - to gather data on it
"""

# ==============================================================================
# Request construction
# ==============================================================================

query_id = 3768636  # SAFE - * - GMD - $SAFE DEX Trades (ref)

df = spice.query(query_id, refresh=True)
query_result = df.to_pandas()  # it's received in a polars dataframe

# get most recent query results using raw sql
# df = spice.query('SELECT * FROM ethereum.blocks LIMIT 5')

# ==============================================================================
# Request handling
# ==============================================================================

# The dex_trades DataFrame will also be used in the other metrics analysis
dex_trades = query_result.copy()

# Convert 'block_date' to datetime - it has to be converted again later when setting index because of the dt.date applied first
dex_trades['block_date'] = pd.to_datetime(dex_trades['block_date']).dt.date
dex_trades = dex_trades.set_index(pd.to_datetime(dex_trades['block_date'])).drop('block_date', axis = 1)

# Isolate only the SAFE numbers of the trades - with addresses of trades

# SAFE Buys
dt_buy = dex_trades.loc[dex_trades['token_bought_symbol'] == 'SAFE'].loc[:, ['blockchain', 'label', 'tx_from', 'token_bought_amount', 'amount_usd']]
dt_buy['type'] = 'buy'
dt_buy = dt_buy.loc[:, ['blockchain', 'label', 'type', 'tx_from', 'token_bought_amount', 'amount_usd']]
dt_buy.columns = ['blockchain', 'label', 'type', 'address', 'amount', 'amount_usd']

# SAFE Sells
dt_sell = dex_trades.loc[dex_trades['token_sold_symbol'] == 'SAFE'].loc[:, ['blockchain', 'label', 'tx_from', 'token_sold_amount', 'amount_usd']]
dt_sell['type'] = 'sell'
dt_sell = dt_sell.loc[:, ['blockchain', 'label', 'type', 'tx_from', 'token_sold_amount', 'amount_usd']]
dt_sell.columns = ['blockchain', 'label', 'type', 'address', 'amount', 'amount_usd']

# Join DFs for complete one - will also be used in the other metrics analysis
dt = pd.concat([dt_buy, dt_sell], axis = 0)

# ==============================================================================
# Ethereum statistics of trades to export to the JT sheet
# ==============================================================================

# Filter by blockchain and trade size wanting to be filtered (defined below in the trading activity analysis)
dt_eth = dt.loc[(dt['blockchain'] == 'ethereum') & (dt['amount'] < 100000)].copy()  # Rationale for 100k SAFE size is explained in Cleaning section below

# Get metrics daily
aggregations = [
    ('trades', 'count'),
    ('min', 'min'),
    ('median', 'median'),
    ('mean', 'mean'),
    ('max', 'max'),
    ('0.05', lambda x: x.quantile(0.05)),
    ('0.90', lambda x: x.quantile(0.90)),
    ('0.95', lambda x: x.quantile(0.95)),
    ('0.98', lambda x: x.quantile(0.98)),
    ('0.99', lambda x: x.quantile(0.99))
]

dt_eth_daily = dt_eth.resample('D')['amount'].agg(aggregations)  # Remember this has removed the 100k trades

# Make sure NaN are treated as 0
dt_eth_daily = dt_eth_daily.fillna(0)

# Rollsing statistics - the percentiles targetted here depend on the analysis below
dt_eth_daily['7d_mean'] = dt_eth_daily['mean'].rolling(window = 7).mean()
dt_eth_daily['14d_mean'] = dt_eth_daily['mean'].rolling(window = 14).mean()
dt_eth_daily['30d_mean'] = dt_eth_daily['mean'].rolling(window = 30).mean()

dt_eth_daily['0.95_30d'] = dt_eth_daily['0.95'].rolling(window = 30).mean()
dt_eth_daily['0.98_30d'] = dt_eth_daily['0.98'].rolling(window = 30).mean()
dt_eth_daily['0.99_30d'] = dt_eth_daily['0.99'].rolling(window = 30).mean()

# Add blockchain
dt_eth_daily['blockchain'] = 'ethereum'

# Add etl_dt (etl datetime for reference of when info was fetched)
dt_eth_daily['etl_dt'] = etl_now
dt_eth_daily['etl_dt'] = dt_eth_daily['etl_dt'].dt.floor('s')

# ==============================================================================
# Gnosis statistics of trades to export to the JT sheet
# ==============================================================================

# Filter by blockchain and trade size wanting to be filtered (defined below in the trading activity analysis)
dt_gno = dt.loc[dt['blockchain'] == 'gnosis'].copy()  # Rationale for 100k SAFE size is explained in Cleaning section below

# Descriptive Statistics
dt_gno.describe()

# Get metrics daily
aggregations = [
    ('trades', 'count'),
    ('min', 'min'),
    ('median', 'median'),
    ('mean', 'mean'),
    ('max', 'max'),
    ('0.05', lambda x: x.quantile(0.05)),
    ('0.90', lambda x: x.quantile(0.90)),
    ('0.95', lambda x: x.quantile(0.95)),
    ('0.98', lambda x: x.quantile(0.98)),
    ('0.99', lambda x: x.quantile(0.99))
]

dt_gno_daily = dt_gno.resample('D')['amount'].agg(aggregations)  # Remember this has removed the 100k trades

# Make sure NaN are treated as 0
dt_gno_daily = dt_gno_daily.fillna(0)

# Rollsing statistics - the percentiles targetted here depend on the analysis below
dt_gno_daily['7d_mean'] = dt_gno_daily['mean'].rolling(window = 7).mean()
dt_gno_daily['14d_mean'] = dt_gno_daily['mean'].rolling(window = 14).mean()
dt_gno_daily['30d_mean'] = dt_gno_daily['mean'].rolling(window = 30).mean()

dt_gno_daily['0.95_30d'] = dt_gno_daily['0.95'].rolling(window = 30).mean()
dt_gno_daily['0.98_30d'] = dt_gno_daily['0.98'].rolling(window = 30).mean()
dt_gno_daily['0.99_30d'] = dt_gno_daily['0.99'].rolling(window = 30).mean()

# Add blockchain
dt_gno_daily['blockchain'] = 'gnosis'

# Add etl_dt (etl datetime for reference of when info was fetched)
dt_gno_daily['etl_dt'] = etl_now
dt_gno_daily['etl_dt'] = dt_gno_daily['etl_dt'].dt.floor('s')

# ==============================================================================
# Combined Data export
# ==============================================================================

dex_trades_exp = pd.concat([dt_eth_daily, dt_gno_daily], axis = 0)

# Open sheet inside workbook
sheet = wb.worksheet('dex_trades')

# Reset index for all data to appear - date is index
dex_trades_exp = dex_trades_exp.reset_index()

# Convert relevant columns in dataframe to str to export - plus additional NaN values to nan in string
dex_trades_exp = dex_trades_exp.astype({'block_date': str, 'etl_dt': str})
dex_trades_exp = dex_trades_exp.fillna('nan')

# DataFrame export
dex_trades_exp_export_list = [dex_trades_exp.columns.tolist()]  # Column labels
dex_trades_exp_export_list.extend(dex_trades_exp.values.tolist())
sheet.update("A1", dex_trades_exp_export_list)

# Export to csv
dex_trades_exp.to_csv('dex_trades.csv')

In [ ]:
dex_trades_exp

In [ ]:
dt_eth_daily

# Other Metrics Analysis

## SAFE/ETH Max Total Weekly Drawdown

In [ ]:
# Uses total_df from market_data section

safe_eth = total_df.loc[total_df['asset'] == 'safe']['close'] / total_df.loc[total_df['asset'] == 'eth']['close']

safe_eth_w = safe_eth / safe_eth.shift(7) - 1  # Moves one row downards
safe_eth_w.describe()  # Looking for max eth_safe depreciation weekly to put the ranges in the Uni V3 pool - we rebalance weekly so we have to aim to cover max possible deppreciation with desired liquidity

In [ ]:
safe_eth_w_df = safe_eth_w.to_frame()

safe_eth_w_df.loc[safe_eth_w_df['close'] == safe_eth_w_df.describe().loc['min','close']]

## Mainnet DEX Trading

In [ ]:
# Uses dex_trades dataframe from dex_trades section

# EXTENDED Analysis of ETHEREUM trades

dt_eth_ext = dt.loc[dt['blockchain'] == 'ethereum']

dt_eth_ext.describe(percentiles=[0.25, 0.50, 0.75, 0.9, 0.95, 0.98, 0.99])

In [ ]:
# Check the distribution of trades to see if there are outliers (I want to target liquidity based on normal data)

# Creating 2x2 subplots with histograms for each column
# fig, axs = plt.subplots(1, 2, figsize=(14, 8))
fig = plt.figure(figsize=(12,6))
ax = fig.add_axes(rect = [1, 1, 1, 1])

ax.hist(dt_eth_ext['amount'], bins=150, edgecolor='black')
ax.set_title('Trade Size Distribution')
ax.set_xlabel('Trade Size')
ax.set_ylabel('Number Of Trades')

# Set tick sizes to see better
ticks = [x for x in range(0, int(dt_eth_ext['amount'].max()), 10000)]
ax.set_xticks(ticks, labels=None)  # Se pasan los ticks en una lista/array
ax.tick_params('x', labelrotation = 45)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Since it's a time series we can also check the average trade size for particular dates

dt_eth_ext_daily = dt_eth_ext.resample('D')['amount'].agg(['mean', 'median', 'max'])

dt_eth_ext_daily[['mean', 'median', 'max']].plot(figsize=(12,6))

plt.tight_layout()
plt.show()

#### Cleaning

In [ ]:
# Clean the dt_ext

# Take out the outliers seen above

# There seems to be only four dates with a maximum above 100,000 SAFE
pprint.pp(dt_eth_ext_daily.loc[dt_eth_ext_daily['max'] > 100000].index.unique().values)
print('')
dt_eth_ext_daily.loc[dt_eth_ext_daily['max'] < 100000].plot(figsize=(12,6))

plt.show()

In [ ]:
# What happens if we only take out the trades above that size

# As a reminder - We want to target 30d mean and the surplus from there to either the 0.95, 0.98, 0.99 rolling 30d mean as well

dt_ = dt.loc[dt['blockchain'] == 'ethereum'].copy()


print('Amount of trades above 100k SAFE:', dt_.loc[dt_['amount'] > 100000].size)
print('Out of a total of:', dt_.size)
print('Trades >100k SAFE over total Trades:', f"{dt_.loc[dt_['amount'] > 100000].size / dt.size * 100}%")

print('')
print('Statistics without the outliers')
dt_.loc[dt_['amount'] < 100000].describe(percentiles=[0.25, 0.50, 0.75, 0.9, 0.95, 0.98, 0.99])

In [ ]:
# Addresses per trading bracket

# Define the percentiles we are interested in
percentiles = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Calculate the percentile values for amount
percentile_values = dt_eth_ext['amount'].quantile(percentiles)

# Dictionary to hold the unique addresses for each percentile range
unique_addresses_by_percentile = {}

# For each percentile, filter the dataframe and find unique 'label'
for p in percentiles:
    threshold = percentile_values[p]
    filtered_df = dt_eth_ext.loc[dt_eth_ext['amount'] >= threshold]
    unique_addresses = filtered_df['address'].nunique()
    unique_addresses_by_percentile[f'{int(p*100)}th Percentile'] = unique_addresses

# Convert the results into a DataFrame for better readability
unique_addresses_df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in unique_addresses_by_percentile.items()])).T

unique_addresses_df

### More Analysis

In [ ]:
dt_eth_ext.resample('D').mean(numeric_only = True)

In [ ]:
"""Total size and value traded
"""

dt_eth_ext.resample('D').sum(numeric_only = True).loc['2024-07':].plot(figsize=(12,6))

plt.show()

## Gnosis Chain DEX Trading

In [ ]:
# Uses dex_trades dataframe from dex_trades section

# EXTENDED Analysis of ETHEREUM trades

dt_gno_ext = dt.loc[dt['blockchain'] == 'gnosis']

dt_gno_ext.describe(percentiles=[0.25, 0.50, 0.75, 0.9, 0.95, 0.98, 0.99])

In [ ]:
# Check the distribution of trades to see if there are outliers (I want to target liquidity based on normal data)

# Creating 2x2 subplots with histograms for each column
# fig, axs = plt.subplots(1, 2, figsize=(14, 8))
fig = plt.figure(figsize=(12,6))
ax = fig.add_axes(rect = [1, 1, 1, 1])

ax.hist(dt_gno_ext['amount'], bins=150, edgecolor='black')
ax.set_title('Trade Size Distribution')
ax.set_xlabel('Trade Size')
ax.set_ylabel('Number Of Trades')

# Set tick sizes to see better
ticks = [x for x in range(0, int(dt_gno_ext['amount'].max()), 500)]
ax.set_xticks(ticks, labels=None)  # Se pasan los ticks en una lista/array
ax.tick_params('x', labelrotation = 45)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Since it's a time series we can also check the average trade size for particular dates

dt_gno_ext_daily = dt_gno_ext.resample('D')['amount'].agg(['mean', 'median', 'max'])

dt_gno_ext_daily[['mean', 'median', 'max']].plot(figsize=(12,6))

plt.tight_layout()
plt.show()

### Cleaning

In [ ]:
# Clean the dt_ext

# Take out the outliers seen above

# There seems to be only four dates with a maximum above 100,000 SAFE
pprint.pp(dt_gno_ext_daily.loc[dt_gno_ext_daily['max'] > 8000].index.unique().values)
print('')
dt_gno_ext_daily.loc[dt_gno_ext_daily['max'] < 8000].plot(figsize=(12,6))

plt.show()

In [ ]:
# What happens if we only take out the trades above that size

# As a reminder - We want to target 30d mean and the surplus from there to either the 0.95, 0.98, 0.99 rolling 30d mean as well

dt_ = dt.loc[dt['blockchain'] == 'gnosis'].copy()


print('Amount of trades above 8k SAFE:', dt_.loc[dt_['amount'] > 8000].size)
print('Out of a total of:', dt_.size)
print('Trades >8k SAFE over total Trades:', f"{dt_.loc[dt_['amount'] > 8000].size / dt.size * 100}%")

print('')
print('Statistics without the outliers')
dt_.loc[dt_['amount'] < 8000].describe(percentiles=[0.25, 0.50, 0.75, 0.9, 0.95, 0.98, 0.99])

## Extra

In [ ]:
dt.loc[dt['amount'] > 10000].loc['2024-09-17':]

# Close Session

In [ ]:
# wb.client.session.close()  # Close the session

# Security Backup (WIP)

Create a csv file backup in Drive for each table

# Archive

In [ ]:
# # ==============================================================================
# # SAFE DEX Data from Dune Analytics
# # ==============================================================================

# query_id = 3777854  # SAFE - * - GMD - $SAFE DEX Trading Volume (Pool)

# headers = {'X-DUNE-API-KEY': config['api_keys']['dune']['key']}

# delay = 10  # time delay in seconds for checking succesfull execution
# q = 0

# # Request for executing the query

# base_url = f"https://api.dune.com/api/v1/query/{query_id}/execute"

# try:
#     r = requests.post(base_url, headers=headers)
#     r.raise_for_status()  # Raise an exception for unsuccessful HTTP status codes

#     execution_id = r.json()['execution_id']  # Get exec id for status check
#     time.sleep(1)

#     print("Request successful!")
# except requests.exceptions.RequestException as e:
#     print("An error occurred:", e)

# # Checking the query execution status

# base_url = f"https://api.dune.com/api/v1/execution/{execution_id}/status"

# while True:
#   r = requests.get(base_url, headers=headers)

#   if r.json()['is_execution_finished']:
#     print("Execution Finished")
#     break

#   print("Awating Execution:", str(q), 's')
#   time.sleep(delay)
#   q += delay

# # Getting the query results - with Dune Client

# dune = DuneClient(
#     api_key = config['api_keys']['dune']['key'],
#     base_url = "https://api.dune.com",
#     request_timeout=(300) # request will time out after 300 seconds
# )

# query_result = dune.get_latest_result_dataframe(
#     query = query_id
#     # # filter for users account more than a month old and more than bottom active tier
#     # , filters="account_age > 30 and fid_active_tier > 1"
#     # # sort result by number of channels they are follow in descending order
#     # , sort_by=["channels desc"]
# )

# # Make date index to resample and SUM data
# query_result['date'] = pd.to_datetime(query_result['block_date'])

# # Daily aggregation of numeric data
# dex_df = query_result.resample('D', kind = 'period', on = 'date').sum(numeric_only = True)

# # Select & Rename columns
# dex_df = dex_df[['trades', 'total_vol']]
# dex_df.columns = ['dex_trades', 'dex_vol']

# # ==============================================================================
# # Joining CG + DEX Data & final things
# # ==============================================================================

# # Add asset col for joining
# dex_df['asset'] = 'safe'

# dex_df

## CoW AMM

- https://docs.cow.fi/cow-protocol/reference/apis/orderbook
- https://explorer.cow.fi/address/0x027e1cbf2c299cba5eb8a2584910d04f1a8aa403

- https://colab.research.google.com/drive/1Si3tBKl004UVxGz9I9spEMUJpc7IMkZs?usp=sharing#scrollTo=8f3iYPODSXbP
- https://colab.research.google.com/drive/1UKUNSMFoOLeg9fy1v-TYnymHSn7N1ghq?usp=sharing#scrollTo=8f3iYPODSXbP

In [ ]:
# import requests
# import pandas as pd

# """
# Get CoW AMM orders (executed and failed)
# """

# account_address = '0x027e1cbf2c299cba5eb8a2584910d04f1a8aa403'

# base_url = f"https://api.cow.fi/mainnet/api/v1/account/{account_address}/orders"